In [1]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [2]:
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [3]:
run_dirs = glob.glob('../runs/*/predict')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 2 runs 

0 ../runs/gan_20190428_1448_gpu=7_bs=4_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=50.0_vr=0.5_is=5_cs=5_rf=10_rs=5_sw=True_glw=[1_100_500_10_0]_clw=[1_1_10]/predict 

1 ../runs/gan_20190430_1720_gpu=7_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=50.0_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_glw=[1_100_500_10_0_0]_clw=[1_1_10]_tag=beefier_decoder_more_summaries_batchnorm/predict 



In [4]:
# set run dir
run_dir = run_dirs[1]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), '*', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    #print(i, os.path.basename(pre_dir))
    print(i, pre_dir.split('/')[-2] + '/' + pre_dir.split('/')[-1])

found 11 pre_dirs
0 eval/gen_090
1 eval/gen_090_02
2 eval/gen_090_04
3 eval/gen_090_08
4 eval/gen_090_12
5 eval/gen_150_04
6 eval/gen_220_04
7 longterm/gen_100
8 test/gen_090_04
9 test/gen_150_04
10 test/gen_220_04


In [9]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[10]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)

print('found meta.csv: {}'.format(csv.shape))

found meta.csv: (271, 41)


In [10]:
# classifier
clf_dir = '../../dx/runs/clf_20190501_0021_gpu=4_bs=64_lr=0.0001_b1=0.9_b2=0.999_ep=0.1_bn=True_mp=True_lk=0.0_ls=[4_2_8_1_8_2_32_1_32_2_128_1_128_1_256_1]_tag=fast_drop/'
model_name = 'clf_0300'
model_file = os.path.join(clf_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')
else:
    print('model file not found!')

found model file


In [2]:
# gpu config
gpu_id = 4

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [12]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/gan_20190430_1720_gpu=7_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=50.0_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_glw=[1_100_500_10_0_0]_clw=[1_1_10]_tag=beefier_decoder_more_summaries_batchnorm/predict/test/gen_220_04/clf_0300.csv


In [14]:
clf_csvs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_csvs = sorted(clf_csvs)

for i, clf_csv in enumerate(clf_csvs):
    print(i, os.path.basename(clf_csv))

0 clf_0300.csv


In [15]:
csv_path = clf_csvs[0]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 44)


In [16]:
def get_metric(csv_in, col, t, metric):

    true_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 3)).sum()
    true_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 2)).sum()
    false_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 2)).sum()
    false_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 3)).sum()
    
    true = true_pos + true_neg
    false = false_pos + false_neg

    if metric == 'f1':
        prec = true_pos / (true_pos + false_pos)
        reca = true_pos / (true_pos + false_neg)

        f1_score = 2 * (prec * reca) / (prec + reca)
        m = f1_score
        
    elif metric == 'acc':
        acc_p = true_pos / (true_pos + false_neg)
        acc_s = true_neg / (true_neg + false_pos)

        balanced_acc = (acc_p + acc_s) / 2
        m = balanced_acc
    
    return (round(m, 3), true, false)

In [17]:
def get_best_threshold(csv_in, col, metric):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        m = get_metric(csv_in, col, t, metric)
        l.append((*m, t))
        
    return max(l)[3]

In [18]:
def get_split_metric(csv_in, col, cv_split, metric):
    
    csv_split = csv_in[csv_in.cv_split == cv_split]
    csv_other = csv_in[csv_in.cv_split != cv_split]
    
    t = get_best_threshold(csv_other, col, metric)
    
    m = get_metric(csv_split, col, t, metric)
    print('split {}: {}'.format(cv_split, m))
    
    return m

In [19]:
def get_cv_metric(csv_in, col, metric):
    l = [get_split_metric(csv_in, col, s, metric)[0] for s in range(5)]
    return np.array(l).mean()

In [20]:
# create 5 folds
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_bins = np.linspace(0, len(pMCI), 6).astype(int)[1:-1]
s_bins = np.linspace(0, len(sMCI), 6).astype(int)[1:-1]

p_splits = np.split(pMCI, p_bins)
s_splits = np.split(sMCI, s_bins)

In [21]:
# write folds to csv
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [22]:
get_cv_metric(csv, 'pAD_xr', 'f1')

split 0: (0.615, 38, 15)
split 1: (0.667, 38, 17)
split 2: (0.647, 41, 12)
split 3: (0.432, 34, 21)
split 4: (0.5, 37, 18)


0.5722

In [23]:
get_cv_metric(csv, 'pAD_yr', 'f1')

split 0: (0.72, 39, 14)
split 1: (0.667, 40, 15)
split 2: (0.64, 35, 18)
split 3: (0.652, 39, 16)
split 4: (0.652, 39, 16)


0.6662000000000001

In [24]:
get_cv_metric(csv, 'pAD_yf', 'f1')

split 0: (0.548, 25, 28)
split 1: (0.548, 27, 28)
split 2: (0.49, 28, 25)
split 3: (0.571, 28, 27)
split 4: (0.571, 31, 24)


0.5456

In [25]:
get_cv_metric(csv, 'pAD_xr', 'acc')

split 0: (0.698, 38, 15)
split 1: (0.725, 38, 17)
split 2: (0.731, 41, 12)
split 3: (0.571, 34, 21)
split 4: (0.625, 37, 18)


0.6699999999999999

In [26]:
get_cv_metric(csv, 'pAD_yr', 'acc')

split 0: (0.783, 39, 14)
split 1: (0.732, 40, 15)
split 2: (0.7, 35, 18)
split 3: (0.718, 39, 16)
split 4: (0.718, 39, 16)


0.7302

In [27]:
get_cv_metric(csv, 'pAD_yf', 'acc')

split 0: (0.642, 31, 22)
split 1: (0.568, 27, 28)
split 2: (0.551, 28, 25)
split 3: (0.568, 33, 22)
split 4: (0.571, 34, 21)


0.5800000000000001